In [ ]:
import cv2
import mss
import pytesseract
import applescript
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib notebook

scptMute = applescript.AppleScript('''
    on mute()
        set volume with output muted
    end mute
    
    on loud()
        set volume without output muted
    end loud
''')
monitor = {"top": 125, "left": 1035, "width": 230, "height": 550}
blacklist = 'Name'
while True: 
    img = np.array(mss.mss().grab(monitor))
    
    #Applying threshold on pixels' Value (or Brightness)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    big = cv2.resize(img, dsize=(0,0), fx=10, fy=10, interpolation=cv2.INTER_LINEAR);
    big[big<180] = 0
    big[big>180] = 255

    results = pytesseract.image_to_data(Image.fromarray(big), output_type='dict') 

    extracted = results['text']

    if any(x==blacklist for x in extracted):
        entry = extracted.index(blacklist)
        x = results["left"][entry] 
        y = results["top"][entry]
        if any(big[y, 0:x]==255):
            '''run applescript'''
            print(blacklist +' quiet now')
            scptMute.call('mute')
    else:
            scptMute.call('loud')


In [ ]:
import cv2
import mss
import pytesseract
import applescript
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.core.debugger import set_trace
%matplotlib notebook

scptMute = applescript.AppleScript('''
    on mute()
        set volume with output muted
    end mute
    
    on loud()
        set volume without output muted
    end loud
''')
# Set global vars
monitor = {"top": 125, "left": 1035, "width": 230, "height": 550}
blacklist = 'Name'
template = None
match_thrsh = 0.8
debug = []
while True: 
    img = np.array(mss.mss().grab(monitor))
    
    #Applying threshold on pixels' Value (or Brightness)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    big = cv2.resize(img, dsize=(0,0), fx=10, fy=10, interpolation=cv2.INTER_LINEAR);
    big[big<180] = 0
    big[big>180] = 255

    results = pytesseract.image_to_data(Image.fromarray(big), output_type='dict') 
    extracted = results['text']
    if template is not None:
        # Template matching should be faster than extracting strings
        match = cv2.matchTemplate(big, template, cv2.TM_CCOEFF_NORMED)
        debug.append(match)
        if np.mean(match) < 0 & any(big[y, 0:x]==255): #this condition needs fixing; how to find a good threshold?
            print(blacklist +' quiet now')
#            set_trace()
            scptMute.call('mute')
        else:    
            scptMute.call('loud')
    elif any(x==blacklist for x in extracted):
        entry = extracted.index(blacklist)
        x = results["left"][entry] 
        y = results["top"][entry]
        width = results["width"][entry]
        height = results["height"][entry]
        if any(big[y, 0:x]==255):
            '''run applescript'''
            print(blacklist +' quiet now')
            scptMute.call('mute')
            template = big[y:y + height, x:x + width]
        else:    
            scptMute.call('loud')
    else:    
        scptMute.call('loud')

    # Press "q" to quit 
    if cv2.waitKey(25) & 0xFF == ord("q"): 
        cv2.destroyAllWindows() 
        break 